<!-- # 1. Define library OpenAI chatcompletion -->

In [ ]:
from openai import OpenAI
import json
import os

token = ''

endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"
# model_name = "Mistral-large"

client = OpenAI(api_key=token, base_url=endpoint)

In [34]:

# # response = client.responses.create(
# #     input=[
# #         {
# #             "role": "system",
# #             "content": "You are a helpful assistant.",
# #         },
# #         {
# #             "role": "user",
# #             "content": "Kể ra các đời vua triều Nguyễn",
# #         }
# #     ],
# #     model=model_name,
# #     stream=True,
# #     # text=
# # )
# response = client.chat.completions.create(
#     messages=[
#         {
#             "role": "system",
#             "content": "You are a helpful assistant.",
#         },
#         {
#             "role": "user",
#             "content": "What is the capital of France?",
#         }
#     ],
#     model=model_name
# )

# print(response.choices[0].message.content)

In [23]:
# response = client.chat.completions.create(
#     messages=[
#         {
#             "role": "system",
#             "content": "You are a helpful assistant.",
#         },
#         {
#             "role": "user",
#             "content": "Cho tôi biết các đời vua triều Nguyễn Việt Nam?",
#         }
#     ],
#     model=model_name,
#     stream=True
# )
# for chunk in response:
#     if chunk.choices[0].delta.content is not None:
#         print(chunk.choices[0].delta.content, end="")

# 2. Extract entity and relasionships via OpenAI + response_format

<!-- # 2. Extract entities and relationships via OpenAI + Output_fomated -->

In [24]:
import json

with open('tma_data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

print(data[:2])

[{'title': 'Chủ tịch tập đoàn TMA', 'content': 'Chủ tịch tập đoàn TMA là chú Nguyễn Hữu Lệ', 'collection_date': '2025-03-14'}, {'title': 'Chi phí phát triển phần mềm tại TMA được tính như thế nào?', 'content': 'Chi phí phát triển phần mềm tại TMA phụ thuộc vào mô hình định giá (theo giờ, theo dự án, mô hình khác); liên hệ qua website tmasolutions để biết thêm chi tiết.', 'collection_date': '2025-03-05'}]


In [25]:
data_content = [item['content'] for item in data]
print(len(data_content))
print(data_content)

745
['Chủ tịch tập đoàn TMA là chú Nguyễn Hữu Lệ', 'Chi phí phát triển phần mềm tại TMA phụ thuộc vào mô hình định giá (theo giờ, theo dự án, mô hình khác); liên hệ qua website tmasolutions để biết thêm chi tiết.', 'Quy trình làm việc tại TMA tuân theo các tiêu chuẩn quốc tế (RUP, ISO 9001:2000, TL 9000, CMMI-Level 5, ISO 27001:2013, Agile…) và kinh nghiệm từ nhiều dự án thành công trong hơn 27 năm, đáp ứng yêu cầu chất lượng và bảo mật từ các tập đoàn lớn.', 'Quy trình tuyển dụng tại TMA gồm: (1) Phỏng vấn vòng 1 → (2) Kiểm tra tiếng Anh (nếu chưa có chứng chỉ) → (3) Phỏng vấn vòng 2 → (4) Thông báo kết quả.', 'Yêu cầu đầu vào thực tập sinh tại TMA: sinh viên năm 3 trở lên, thực tập fulltime tối thiểu 3 tháng; hồ sơ gồm CV Tiếng Anh, Bảng điểm/Bằng cấp (nếu có) và Giấy giới thiệu thực tập.', 'Điểm tiếng Anh đầu vào của TMA yêu cầu TOEIC ≥ 550 hoặc chứng chỉ tương đương.', 'Công viên sáng tạo TMA Bình Định vào năm 2017: TMA quyết định xây dựng Công viên Sáng tạo TMA Bình Định tại Thung

In [ ]:
system_prompt = """
Trích xuất các thực thể (nodes) và mối quan hệ giữa chúng (edges) từ văn bản đã cho bằng tiếng Việt.  
- Các thực thể và mối quan hệ phải được nhận diện và trình bày rõ ràng bằng tiếng Việt có dấu đầy đủ.  
- Đảm bảo rằng:
    + {{entity}} luôn được viết **in thường**, 
    + {{ENTITY_TYPE}} và {{RELATIONSHIP_TYPE}} luôn được viết **IN HOA** và các từ trong tên được nối với nhau bằng dấu gạch dưới ("_").  
- Trích xuất đầy đủ cả các mối quan hệ **trực tiếp** và **gián tiếp** (ngầm hiểu từ ngữ cảnh).  
- Chỉ định rõ loại thực thể và loại mối quan hệ phù hợp với ngữ cảnh.  

###**Quy tắc:**  
1. entity: 
- Tên thực thể phải là các danh từ hoặc cụm danh từ, viết in thường và nối với nhau bằng dấu gạch dưới (`_`).
- Ví dụ: tma, nguyễn hữu lệ, tma bình định,...
2. ENTITY_TYPE:
- Loại thực thể phải được xác định rõ ràng và viết IN HOA, nối các từ bằng dấu gạch dưới (`_`).
- Ví dụ: TẬP_ĐOÀN, CHI_PHÍ, NHÂN_VIÊN, CHỦ_TỊCH, GIÁM_ĐỐC, THÀNH_VIÊN, DỰ_ÁN, GIẢI_PHÁP, LĨNH_VỰC, CÔNG_NGHỆ...  
3. RELATIONSHIP_TYPE: 
- Loại mối quan hệ phải mô tả hành động hoặc trạng thái giữa các thực thể, viết IN HOA và nối các từ bằng dấu gạch dưới (`_`)
- Ví dụ: THUỘC, PHỤ_THUỘC, NHÂN_VIÊN_CỦA, LÀ, ...  
4. Đảm bảo định dạng đầu ra theo mẫu sau:  

###**Định dạng:**  
Entities:
{{entity1}}: {{ENTITY_TYPE}}  
{{entity2}}: {{ENTITY_TYPE}}  
...  

Relationships:
({{entity1}}, {{RELATIONSHIP_TYPE}}, {{entity2}})  
({{entity3}}, {{RELATIONSHIP_TYPE}}, {{entity4}})  
...  

###**Ví dụ:**  
Input:
"Chi phí phát triển phần mềm tại TMA phụ thuộc vào mô hình định giá khác nhau như: theo giờ, theo dự án, hay các mô hình khác."  

Output:
Entities: 
- tma: TẬP_ĐOÀN  
- chi_phí_phát_triển_phần_mềm: CHI_PHÍ  
- mô_hình_định_giá: MÔ_HÌNH  
- theo_giờ: LOẠI_MÔ_HÌNH  
- theo_dự_án: LOẠI_MÔ_HÌNH  
- các_mô_hình_khác: LOẠI_MÔ_HÌNH  

Relationships: 
- (tma, PHÁT_TRIỂN, chi_phí_phát_triển_phần_mềm)  
- (chi_phí_phát_triển_phần_mềm, PHỤ_THUỘC, mô_hình_định_giá)  
- (mô_hình_định_giá, BAO_GỒM, theo_giờ)  
- (mô_hình_định_giá, BAO_GỒM, theo_dự_án)  
- (mô_hình_định_giá, BAO_GỒM, các_mô_hình_khác)  
"""

In [27]:
# u_message = '###Input: '
u_message = data_content[0]
u_message

'Chủ tịch tập đoàn TMA là chú Nguyễn Hữu Lệ'

In [28]:
from pydantic import BaseModel

class Extract_Entity(BaseModel):
    name: str
    type: str

class Relationship(BaseModel):
    entity1: Extract_Entity
    entity2: Extract_Entity
    relationship: str

class Extract_Entity_Relationship(BaseModel):
    entities: list[Extract_Entity]
    relationships: list[Relationship]

In [60]:
# response = client.beta.chat.completions.parse(
#     model=model_name,
#     messages=[
#         {"role": "system", "content": system_prompt},
#         {"role": "system", "content": "Nguyễn Hữu Lệ là chủ tịch của TMA"}
#     ],
#     response_format=Extract_Entity_Relationship
# )

In [ ]:
# print(response.choices[0].message.parsed)

entities=[Extract_Entity(name='nguyễn_hữu_lệ', type='CHỦ_TỊCH'), Extract_Entity(name='tma', type='TẬP_ĐOÀN')] relationships=[Relationship(entity1=Extract_Entity(name='nguyễn_hữu_lệ', type='CHỦ_TỊCH'), entity2=Extract_Entity(name='tma', type='TẬP_ĐOÀN'), relationship='LÀ_CHỦ_TỊCH_CỦA')]


In [ ]:
# entities = response.choices[0].message.parsed.entities
# relationships = response.choices[0].message.parsed.relationships
# print("Entities Defined")
# for entity in entities:
#     print(f'MERGE ({entity.name}:{entity.type} {{name: "{entity.name}"}})')

# print("\n\nRelationship Defined")
# for relationship in relationships:
#     print(f'MERGE ({relationship.entity1.name})-[:{relationship.relationship}]->({relationship.entity2.name})')

Entities Defined
MERGE (nguyễn_hữu_lệ:CHỦ_TỊCH {name: "nguyễn_hữu_lệ"})
MERGE (tma:CÔNG_TY {name: "tma"})


Relationship Defined
MERGE (nguyễn_hữu_lệ)-[:LÀ_CHỦ_TỊCH_CỦA]->(tma)


<!-- ## 2.1 Create function 4 Build KG -->

In [ ]:
from tqdm import tqdm

def extract_entity_relationship(docs):
    for doc in tqdm(docs, total=(len(docs)), desc='Build KG processing: '):
        entities_set = set()
        relationships_set = set()  
        response = client.beta.chat.completions.parse(
            model=model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": doc}
            ],
            response_format=Extract_Entity_Relationship
        )
        entities = response.choices[0].message.parsed.entities
        relationships = response.choices[0].message.parsed.relationships
        for entity in entities:
            entities_set.add(f'MERGE ({entity.name}:{entity.type} {{name: "{entity.name}"}})')
        for relationship in relationships:
            relationships_set.add(f'MERGE ({relationship.entity1.name})-[:{relationship.relationship}]->({relationship.entity2.name})')

        # Ghi vào file ngay sau mỗi doc
        with open("KG_BUILDed.txt", "a", encoding="utf-8") as file:
            file.write('\n'.join(entities_set) + '\n')
            file.write('\n')
            file.write('\n'.join(relationships_set) + '\n\n\n')
    
    return list(entities_set), list(relationships_set)
    

In [ ]:
# print(len(data_content))

745


In [30]:
# data_processing = data_content[0:20]
# print(len(data_processing))

In [31]:
# entities_lst, relationships_lst = extract_entity_relationship(data_processing)

In [ ]:
# data_processing = data_content[20:21]
# data_processing = data_content[21:22]
# data_processing = data_content[34:35]
# data_processing = data_content[35:55]
# data_processing = data_content[55:75]
# data_processing = data_content[75:82]
data_processing = data_content[82:132]      # Dung day1
# data_processing = data_content[132:182]

In [62]:
print(len(data_processing))

50


In [58]:
print(len(data_processing))
entities_lst, relationships_lst = extract_entity_relationship(data_processing)

50


Build KG processing:   0%|          | 0/50 [00:01<?, ?it/s]


UnprocessableEntityError: Error code: 422 - {'error': {'code': 'Invalid input', 'status': 422, 'message': 'invalid input error', 'details': [{'type': 'value_error', 'loc': ['body', 'response_format', 'type'], 'msg': "Value error, Response format was json_schema but must be either 'text' or 'json_object'.", 'input': 'json_schema', 'ctx': {'error': {}}}]}}

<!-- # 2.2 Remove duplicate in KG txt -->

# 3. Remove duplicate Entities in output

In [63]:
def remover_merge_duplicate(file_path):
    seen = set()  # Dùng để theo dõi các lệnh đã gặp
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
        print(lines)
    
    output_file = "KG_Builded_RmDuplicate.txt"
    with open(output_file, "w", encoding="utf-8") as file:
        for line in lines:
            line = line.strip()
            if line and line not in seen:  
                seen.add(line)  
                file.write(line + '\n')  

    print(f"✅ Đã loại bỏ dòng trùng lặp, kết quả lưu tại: {output_file}")


In [ ]:
# remover_merge_duplicate("KG_BUILDed.txt")

['MERGE (tma:TẬP_ĐOÀN {name: "tma"})\n', 'MERGE (nguyễn_hữu_lệ:CHỦ_TỊCH {name: "nguyễn_hữu_lệ"})\n', '\n', 'MERGE (nguyễn_hữu_lệ)-[:LÀ_CHỦ_TỊCH_CỦA]->(tma)\n', '\n', '\n', 'MERGE (website_tmasolutions:WEBSITE {name: "website_tmasolutions"})\n', 'MERGE (chi_phí_phát_triển_phần_mềm:CHI_PHÍ {name: "chi_phí_phát_triển_phần_mềm"})\n', 'MERGE (theo_dự_án:LOẠI_MÔ_HÌNH {name: "theo_dự_án"})\n', 'MERGE (mô_hình_định_giá:MÔ_HÌNH {name: "mô_hình_định_giá"})\n', 'MERGE (theo_giờ:LOẠI_MÔ_HÌNH {name: "theo_giờ"})\n', 'MERGE (tma:TẬP_ĐOÀN {name: "tma"})\n', 'MERGE (mô_hình_khác:LOẠI_MÔ_HÌNH {name: "mô_hình_khác"})\n', '\n', 'MERGE (chi_phí_phát_triển_phần_mềm)-[:PHỤ_THUỘC]->(mô_hình_định_giá)\n', 'MERGE (tma)-[:PHÁT_TRIỂN]->(chi_phí_phát_triển_phần_mềm)\n', 'MERGE (mô_hình_định_giá)-[:BAO_GỒM]->(mô_hình_khác)\n', 'MERGE (website_tmasolutions)-[:LIÊN_HỆ_CỦA]->(tma)\n', 'MERGE (mô_hình_định_giá)-[:BAO_GỒM]->(theo_giờ)\n', 'MERGE (mô_hình_định_giá)-[:BAO_GỒM]->(theo_dự_án)\n', '\n', '\n', 'MERGE (chú_ng

# 4. Connect with Neo4j python via Langchain

In [1]:
# !pip install langchain-neo4j

  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl (161 kB)
  Using cached charset_normalizer-3.4.1-cp310-cp310-win_amd64.whl (102 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)


You should consider upgrading via the 'D:\.thuctap\tma\code\graph\build_graph_tma_info\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [38]:
from langchain_neo4j import Neo4jGraph
import os

os.environ['NEO4J_URI'] = 'bolt://localhost:7687'
os.environ['NEO4J_USERNAME'] = 'neo4j'
os.environ['NEO4J_PASSWORD'] = '123456789'
# kg = Neo4jGraph(url='bolt://localhost:7687', username='neo4j', password='123456789', database='neo4j')


In [39]:
kg = Neo4jGraph()

In [42]:
kg.query(
    """
        MERGE (tma:TẬP_ĐOÀN {name: "tma"})
        MERGE (nguyễn_hữu_lệ:CHỦ_TỊCH {name: "nguyễn_hữu_lệ"})

        MERGE (nguyễn_hữu_lệ)-[:LÀ_CHỦ_TỊCH_CỦA]->(tma)


        MERGE (website_tmasolutions:WEBSITE {name: "website_tmasolutions"})
        MERGE (chi_phí_phát_triển_phần_mềm:CHI_PHÍ {name: "chi_phí_phát_triển_phần_mềm"})
        MERGE (theo_dự_án:LOẠI_MÔ_HÌNH {name: "theo_dự_án"})
        MERGE (mô_hình_định_giá:MÔ_HÌNH {name: "mô_hình_định_giá"})
        MERGE (theo_giờ:LOẠI_MÔ_HÌNH {name: "theo_giờ"})
        MERGE (mô_hình_khác:LOẠI_MÔ_HÌNH {name: "mô_hình_khác"})

        MERGE (chi_phí_phát_triển_phần_mềm)-[:PHỤ_THUỘC]->(mô_hình_định_giá)
        MERGE (tma)-[:PHÁT_TRIỂN]->(chi_phí_phát_triển_phần_mềm)
        MERGE (mô_hình_định_giá)-[:BAO_GỒM]->(mô_hình_khác)
        MERGE (website_tmasolutions)-[:LIÊN_HỆ_CỦA]->(tma)
        MERGE (mô_hình_định_giá)-[:BAO_GỒM]->(theo_giờ)
        MERGE (mô_hình_định_giá)-[:BAO_GỒM]->(theo_dự_án)
"""
)

[]

In [55]:
kg.query("MATCH (n)-[r]->(n2) RETURN n,r, n2")

[{'n': {'name': 'tma'},
  'r': ({'name': 'tma'},
   'PHÁT_TRIỂN',
   {'name': 'chi_phí_phát_triển_phần_mềm'}),
  'n2': {'name': 'chi_phí_phát_triển_phần_mềm'}},
 {'n': {'name': 'nguyễn_hữu_lệ'},
  'r': ({'name': 'nguyễn_hữu_lệ'}, 'LÀ_CHỦ_TỊCH_CỦA', {'name': 'tma'}),
  'n2': {'name': 'tma'}},
 {'n': {'name': 'website_tmasolutions'},
  'r': ({'name': 'website_tmasolutions'}, 'LIÊN_HỆ_CỦA', {'name': 'tma'}),
  'n2': {'name': 'tma'}},
 {'n': {'name': 'chi_phí_phát_triển_phần_mềm'},
  'r': ({'name': 'chi_phí_phát_triển_phần_mềm'},
   'PHỤ_THUỘC',
   {'name': 'mô_hình_định_giá'}),
  'n2': {'name': 'mô_hình_định_giá'}},
 {'n': {'name': 'mô_hình_định_giá'},
  'r': ({'name': 'mô_hình_định_giá'}, 'BAO_GỒM', {'name': 'theo_dự_án'}),
  'n2': {'name': 'theo_dự_án'}},
 {'n': {'name': 'mô_hình_định_giá'},
  'r': ({'name': 'mô_hình_định_giá'}, 'BAO_GỒM', {'name': 'theo_giờ'}),
  'n2': {'name': 'theo_giờ'}},
 {'n': {'name': 'mô_hình_định_giá'},
  'r': ({'name': 'mô_hình_định_giá'}, 'BAO_GỒM', {'name': 

## 4.1. Connect with Neo4j python via Neo4j directly

In [5]:
# !pip install neo4j

You should consider upgrading via the 'D:\.thuctap\tma\code\graph\build_graph_tma_info\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [44]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "123456789")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [56]:
records, summary, keys = driver.execute_query(
    "MATCH (n)-[r]->(n2) RETURN n,r, n2",
    # database_="neo4j",
)

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after,
))

The query `MATCH (n)-[r]->(n2) RETURN n,r, n2` returned 7 records in 2 ms.


C:\Users\Admin\AppData\Local\Temp\ipykernel_22208\834041331.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


In [57]:
records, summary, keys

([<Record n=<Node element_id='4:43fc73d6-4b97-49b7-9836-4448038922e8:0' labels=frozenset({'TẬP_ĐOÀN'}) properties={'name': 'tma'}> r=<Relationship element_id='5:43fc73d6-4b97-49b7-9836-4448038922e8:1152923703630102528' nodes=(<Node element_id='4:43fc73d6-4b97-49b7-9836-4448038922e8:0' labels=frozenset({'TẬP_ĐOÀN'}) properties={'name': 'tma'}>, <Node element_id='4:43fc73d6-4b97-49b7-9836-4448038922e8:3' labels=frozenset({'CHI_PHÍ'}) properties={'name': 'chi_phí_phát_triển_phần_mềm'}>) type='PHÁT_TRIỂN' properties={}> n2=<Node element_id='4:43fc73d6-4b97-49b7-9836-4448038922e8:3' labels=frozenset({'CHI_PHÍ'}) properties={'name': 'chi_phí_phát_triển_phần_mềm'}>>,
  <Record n=<Node element_id='4:43fc73d6-4b97-49b7-9836-4448038922e8:1' labels=frozenset({'CHỦ_TỊCH'}) properties={'name': 'nguyễn_hữu_lệ'}> r=<Relationship element_id='5:43fc73d6-4b97-49b7-9836-4448038922e8:1152921504606846977' nodes=(<Node element_id='4:43fc73d6-4b97-49b7-9836-4448038922e8:1' labels=frozenset({'CHỦ_TỊCH'}) prop

# 5. Generate Cypher query 4 GraphDB

In [ ]:
# CALL db.labels();
# CALL db.relationshipTypes();
# CALL db.propertyKeys();


In [ ]:
schema = """
- Relationship Types: LÀ_CHỦ_TỊCH_CỦA, PHỤ_THUỘC, PHÁT_TRIỂN, BAO_GỒM, LIÊN_HỆ_CỦA
- Labels: TẬP_ĐOÀN, CHỦ_TỊCH, WEBSITE, CHI_PHÍ, LOẠI_MÔ_HÌNH, MÔ_HÌNH
- Property Keys: name
"""
question = 'Chủ tịch tập đoàn TMA là ai?'

CYPHER_GENERATION_TEMPLATE = f"""
Nhiệm vụ: Hãy tạo ra câu lệnh Cypher để truy vấn trong Graph Database theo hướng dẫn dưới đây mà không cần phân tích.
Hướng dẫn:
1. Phân tích câu hỏi và xác định các thành phần sau:
    - Relationships (Relationship Types).
    - Nodes (Labels).
2. Câu lệnh Cypher trả về kết quả phải hợp lệ và đúng cú pháp với các giá trị property luôn là chữ "in thường" và relationships luôn là chữ "IN HOA".
3. Chỉ sử dụng các thành phần từ Schema được cung cấp bên dưới để tạo ra câu lệnh Cypher.
- Schema cung cấp:{schema}

Ví dụ:
# Chủ tịch tập đoàn TMA là ai?
MATCH (n)-[:LÀ_CHỦ_TỊCH_CỦA]->(TMA: TẬP_ĐOÀN)
WHERE TMA.name='tma'
RETURN n.name

# Chi phí phát triển phần mềm tại TMA là gì?
MATCH (chi_phí_phát_triển_phần_mềm)-[:PHỤ_THUỘC]->(mô_hình_định_giá)-[:BAO_GỒM]->(n:LOẠI_MÔ_HÌNH)
RETURN n.name

# # TMA có bao nhiêu kỹ sư?
# MATCH (tma)-[:SỞ_HỮU]->(n)
# WHERE tma.name='tma'
# RETURN n.name

Câu hỏi:
{question}
"""

In [112]:
print(CYPHER_GENERATION_TEMPLATE)


Nhiệm vụ: Hãy tạo ra câu lệnh Cypher để truy vấn trong Graph Database theo hướng dẫn dưới đây mà không cần phân tích.
Hướng dẫn:
1. Phân tích câu hỏi và xác định các thành phần sau:
    - Relationships (Relationship Types).
    - Nodes (Labels).
2. Câu lệnh Cypher trả về kết quả phải hợp lệ và đúng cú pháp với các giá trị property luôn là chữ "in thường" và relationships luôn là chữ "IN HOA".
3. Chỉ sử dụng các thành phần từ Schema được cung cấp bên dưới để tạo ra câu lệnh Cypher.
- Schema cung cấp:
- Relationship Types: LÀ_CHỦ_TỊCH_CỦA, PHỤ_THUỘC, PHÁT_TRIỂN, BAO_GỒM, LIÊN_HỆ_CỦA
- Labels: TẬP_ĐOÀN, CHỦ_TỊCH, WEBSITE, CHI_PHÍ, LOẠI_MÔ_HÌNH, MÔ_HÌNH
- Property Keys: name


Ví dụ:
# Chủ tịch tập đoàn TMA là ai?
MATCH (n)-[:LÀ_CHỦ_TỊCH_CỦA]->(TMA: TẬP_ĐOÀN)
WHERE TMA.name='tma'
RETURN n.name

# Chi phí phát triển phần mềm tại TMA là gì?
MATCH (chi_phí_phát_triển_phần_mềm)-[:PHỤ_THUỘC]->(mô_hình_định_giá)-[:BAO_GỒM]->(n:LOẠI_MÔ_HÌNH)
RETURN n.name

# TMA có bao nhiêu kỹ sư?
MATCH (tma)-[

In [97]:
a = kg.query("""
MATCH (c:CHỦ_TỊCH)-[:LÀ_CHỦ_TỊCH_CỦA]->(t:TẬP_ĐOÀN)  
RETURN c
""")

# 6. Fuzzy matching 

In [114]:
!pip install rapidfuzz

You should consider upgrading via the 'D:\.thuctap\tma\code\graph\build_graph_tma_info\myenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [150]:
from rapidfuzz import process 

closest_matchs = process.extract('Tôi_đi_học',['Tôi_đi_học!', 'Tôi_đi_chơi', 'Tôi_đi', 'Tôi_học'])
closest_matchs
# closest_match, score, index = process.extractOne('Tôi_đi_học',['Tôi_đi_học', 'Tôi_đi_chơi', 'Tôi_đi', 'Tôi_học'])
# closest_match, score, index

[('Tôi_đi_học!', 95.23809523809523, 0),
 ('Tôi_đi', 90.0, 2),
 ('Tôi_học', 82.35294117647058, 3),
 ('Tôi_đi_chơi', 76.19047619047619, 1)]

In [146]:
def find_closest_entity(lst_entities: list[str], entity_extract: str):
    closest_match, score, index = process.extractOne('Tôi_đi_học',['Tôi_đi_học', 'Tôi_đi_chơi', 'Tôi_đi', 'Tôi_học'])
    return closest_match

In [148]:
a = find_closest_entity('Tôi_đi_học',['Tôi_đi_học', 'Tôi_đi_chơi', 'Tôi_đi', 'Tôi_học'])

In [149]:
print(a)

Tôi_đi_học


In [ ]:
query_entities = entities_lst
matches = find_closest_entity(query_entities, node_mapping)
